![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Introduction to Redis Python

This notebook introduces [Redis](https://redis.io) and the standard Python client, [redis-py](https://redis-py.readthedocs.io/en/stable/), for interacting with the database. We will explore the basics of Redis setup, data structures, and capabilities like vector search!

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/redis-intro/00_redis_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Environment Setup

### Install Python Dependencies

In [1]:
%pip install -q redis pandas


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [2]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Hello World Redis

Now let's connect to the Redis db and get a basic feel for the most common
commands and data structures.

In [3]:
import redis
import json
import numpy as np

from time import sleep

# Connect with the Redis Python Client
client = redis.Redis.from_url(REDIS_URL)

client.ping()

True

In [4]:
client.dbsize() # should be empty

0

Redis, at it's core, is a simple key/value store. It supports a number of interesting
and flexible data structures that can solve a variatey of business and operational
problems.

### Strings

The basic string data type can be accessed using set/get methods. You can also place a
TTL policy (expiration) on any key in Redis.

In [5]:
client.set("hello", "world")

True

In [6]:
client.get("hello")

b'world'

In [7]:
client.delete("hello")

1

In [8]:
client.set("hello", "world")
client.expire("hello", time=3)

sleep(4)

# should be EMPTY
client.get("hello")

### Hashes
Hashes are collections of key/value pairs that are grouped together. It gets
serialized as a string in Redis, but can hold a variety of data in each field.

You can think of a Hash as a one-level deep Python dictionary.


In [9]:
obj = {
    "user": "john",
    "age": 45,
    "job": "dentist",
    "bio": "long form text of john's bio",
    "user_embedding": np.array([0.3, 0.4, -0.8], dtype=np.float32).tobytes() # cast vectors to bytes string
}

In [10]:
client.hset("user:john", mapping=obj)

5

In [11]:
client.hgetall("user:john")

{b'user': b'john',
 b'age': b'45',
 b'job': b'dentist',
 b'bio': b"long form text of john's bio",
 b'user_embedding': b'\x9a\x99\x99>\xcd\xcc\xcc>\xcd\xccL\xbf'}

In [12]:
client.delete("user:john")

1

### JSON
With the JSON capabilitie enabled, Redis can be a drop-in replacement for MongoDB
or other slower document databases. You can store nested and structured JSON data
directly in Redis.

In [13]:
# set a JSON obj
obj = {
    "user": "john",
    "metadata": {
        "age": 45,
        "job": "dentist",
    },
    "user_embedding": [0.3, 0.4, -0.8]
}

client.json().set("user:john", "$", obj)

True

In [14]:
# get user JSON obj
client.json().get("user:john")

{'user': 'john',
 'metadata': {'age': 45, 'job': 'dentist'},
 'user_embedding': [0.3, 0.4, -0.8]}

In [15]:
# grab array length for embedding field
client.json().arrlen("user:john", "$.user_embedding")

[3]

In [16]:
# grab obj keys
client.json().objkeys("user:john", "$")

[[b'user', b'metadata', b'user_embedding']]

In [17]:
# delete user JSON
client.delete("user:john")

1

### Lists
Lists store sequences of information... potentially list of messages in an LLM
converstion flow, or really any list of items in a queue.

In [18]:
# add items to a list
client.rpush("messages:john", *[
    json.dumps({"role": "user", "content": "Hello what can you do for me?"}),
    json.dumps({"role": "assistant", "content": "Hi, I am a helpful virtual assistant."})
])

2

In [19]:
# list all items in the list using indices
[json.loads(msg) for msg in client.lrange("messages:john", 0, -1)]

[{'role': 'user', 'content': 'Hello what can you do for me?'},
 {'role': 'assistant', 'content': 'Hi, I am a helpful virtual assistant.'}]

In [20]:
# count items in the list
client.llen("messages:john")

2

In [21]:
# pop the first item from the list and push to another list
client.rpoplpush("messages:john", "read_messages:john")

b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}'

In [22]:
client.lrange("read_messages:john", 0, -1)

[b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}']

In [23]:
# list cleanup
client.delete("messages:john", "read_messages:john")

2

### Pipelines
All Redis commands can be pipelined to gain some round trip latency improvements.

In [24]:
with client.pipeline(transaction=False) as pipe:
    for i in range(50):
        pipe.json().set(f"user:{i}", "$", obj)
    # execute batch
    pipe.execute()

In [25]:
client.dbsize()

50

In [26]:
# clean up!
client.flushall()

True